# Lab 4.1: Introduction to TensorFlow - Why Frameworks Matter

## Duration: 45 minutes

## Learning Objectives
By the end of this lab, you will be able to:
- Understand the key benefits of using TensorFlow/Keras over NumPy
- Build your first neural network with TensorFlow in just a few lines
- Compare manual NumPy implementation with TensorFlow's automatic features
- Appreciate automatic differentiation and GPU acceleration
- Use built-in optimizers and loss functions

## Prerequisites
- Completion of Labs 1-3 (NumPy neural network implementation)
- Basic understanding of neural networks
- Python and NumPy knowledge

## Key Concepts
- **TensorFlow/Keras**: Google's deep learning framework with high-level API
- **Automatic Differentiation**: No manual backpropagation needed!
- **Built-in Functions**: Optimizers, losses, metrics, and more
- **GPU Acceleration**: Automatic hardware optimization
- **Model Management**: Easy saving, loading, and deployment

## Setup and Imports

First, let's import our libraries and see what we're working with:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Welcome to TensorFlow!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print()

# Check hardware availability
if tf.config.list_physical_devices('GPU'):
    print("GPU is available - your models will run faster!")
    gpu_devices = tf.config.list_physical_devices('GPU')
    for device in gpu_devices:
        print(f"   Found GPU: {device.name}")
else:
    print("Using CPU (still much easier than NumPy!)")

print("\n" + "="*60)
print("Ready to see why TensorFlow makes deep learning easier?")
print("="*60)

## Step 1: The Pain of NumPy - Manual Implementation

Remember all the code we wrote in Labs 1-3? Let's remind ourselves of the complexity:

In [ ]:
# This is what we had to write in NumPy for a simple neural network!
class SimpleNumpyNetwork:
    """A reminder of our NumPy implementation from earlier labs"""
    
    def __init__(self, layer_sizes):
        self.layer_sizes = layer_sizes
        self.parameters = {}
        self.gradients = {}
        
        # Initialize weights and biases manually
        for i in range(1, len(layer_sizes)):
            self.parameters[f'W{i}'] = np.random.randn(layer_sizes[i], layer_sizes[i-1]) * 0.01
            self.parameters[f'b{i}'] = np.zeros((layer_sizes[i], 1))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, a):
        return a * (1 - a)
    
    def forward(self, X):
        # Manual forward propagation
        self.cache = {'A0': X}
        A = X
        
        for i in range(1, len(self.layer_sizes)):
            Z = self.parameters[f'W{i}'] @ A + self.parameters[f'b{i}']
            A = self.sigmoid(Z)
            self.cache[f'Z{i}'] = Z
            self.cache[f'A{i}'] = A
        
        return A
    
    def backward(self, X, Y, output):
        # Manual backpropagation - so much code!
        m = X.shape[1]
        L = len(self.layer_sizes) - 1
        
        # Output layer gradient
        dA = output - Y
        
        for i in reversed(range(1, L + 1)):
            dZ = dA * self.sigmoid_derivative(self.cache[f'A{i}'])
            self.gradients[f'W{i}'] = (1/m) * dZ @ self.cache[f'A{i-1}'].T
            self.gradients[f'b{i}'] = (1/m) * np.sum(dZ, axis=1, keepdims=True)
            if i > 1:
                dA = self.parameters[f'W{i}'].T @ dZ
    
    def update_parameters(self, learning_rate):
        # Manual parameter updates
        for i in range(1, len(self.layer_sizes)):
            self.parameters[f'W{i}'] -= learning_rate * self.gradients[f'W{i}']
            self.parameters[f'b{i}'] -= learning_rate * self.gradients[f'b{i}']
    
    def train_step(self, X, Y, learning_rate=0.01):
        # One training step
        output = self.forward(X)
        loss = -np.mean(Y * np.log(output + 1e-8) + (1 - Y) * np.log(1 - output + 1e-8))
        self.backward(X, Y, output)
        self.update_parameters(learning_rate)
        return loss

# Create a simple dataset
np.random.seed(42)
X_train = np.random.randn(2, 100)
Y_train = (X_train[0] + X_train[1] > 0).astype(int).reshape(1, -1)

print("NumPy Implementation:")
print("="*40)
print("Lines of code for basic neural network: ~70")
print("Features we had to implement manually:")
print("  X Forward propagation")
print("  X Backpropagation")
print("  X Activation functions")
print("  X Loss functions")
print("  X Parameter updates")
print("  X Weight initialization")
print("\nLet's train it briefly...")

# Train the NumPy network
numpy_net = SimpleNumpyNetwork([2, 4, 1])
numpy_losses = []

start_time = time.time()
for epoch in range(100):
    loss = numpy_net.train_step(X_train, Y_train, learning_rate=0.1)
    numpy_losses.append(loss)
    if epoch % 20 == 0:
        print(f"  Epoch {epoch}: Loss = {loss:.4f}")
numpy_time = time.time() - start_time

print(f"\nTraining time: {numpy_time:.3f} seconds")
print(f"Final loss: {numpy_losses[-1]:.4f}")

## Step 2: The Magic of TensorFlow - Same Network, 10x Easier!

Now let's build the EXACT same network with TensorFlow/Keras:

In [ ]:
print("TensorFlow Implementation:")
print("="*40)

# Build the same network in TensorFlow - SO MUCH SIMPLER!
tf_model = keras.Sequential([
    layers.Dense(4, activation='sigmoid', input_shape=(2,)),
    layers.Dense(1, activation='sigmoid')
])

# Compile with optimizer and loss - one line!
tf_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Neural network created in just 6 lines!")
print()
print("What TensorFlow handles for us:")
print("  ✓ Forward propagation (automatic)")
print("  ✓ Backpropagation (automatic)")
print("  ✓ Activation functions (30+ built-in)")
print("  ✓ Loss functions (20+ built-in)")
print("  ✓ Optimizers (10+ built-in, including Adam!)")
print("  ✓ Weight initialization (smart defaults)")
print("  ✓ GPU acceleration (automatic)")
print("  ✓ Batch processing (automatic)")

print("\nLet's train it...")

# Prepare data (TensorFlow format)
X_train_tf = X_train.T  # TensorFlow expects (samples, features)
Y_train_tf = Y_train.T

# Train the model - one line!
start_time = time.time()
history = tf_model.fit(
    X_train_tf, Y_train_tf,
    epochs=100,
    batch_size=32,
    verbose=0  # Suppress output for cleaner display
)
tf_time = time.time() - start_time

print(f"\nTraining time: {tf_time:.3f} seconds")
print(f"Final loss: {history.history['loss'][-1]:.4f}")
print(f"Final accuracy: {history.history['accuracy'][-1]:.2%}")

# Visualize the comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Loss comparison
ax1.plot(numpy_losses, label='NumPy Implementation', linewidth=2, color='blue')
ax1.plot(history.history['loss'], label='TensorFlow', linewidth=2, color='red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Implementation comparison
categories = ['Lines of\nCode', 'Training\nSpeed', 'Features', 'GPU\nSupport']
numpy_scores = [70, 30, 20, 0]  # Relative scores
tf_scores = [6, 90, 100, 100]   # Relative scores

x = np.arange(len(categories))
width = 0.35

ax2.bar(x - width/2, numpy_scores, width, label='NumPy', color='blue', alpha=0.7)
ax2.bar(x + width/2, tf_scores, width, label='TensorFlow', color='red', alpha=0.7)
ax2.set_ylabel('Score (Higher is Better)')
ax2.set_title('Implementation Comparison')
ax2.set_xticks(x)
ax2.set_xticklabels(categories)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nKey Insight: TensorFlow gives us the same (or better) results")
print("   with 10x less code and many more features!")

## Step 3: Putting It All Together - A Complete Example

Let's build a complete classification model with all of TensorFlow's benefits:

In [ ]:
print("Complete TensorFlow Workflow:")
print("="*50)

# Generate a multi-class classification dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Create dataset
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=3,
    random_state=42
)

# Preprocess
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Dataset: {X_train.shape[0]} training, {X_test.shape[0]} test samples")
print(f"Features: {X_train.shape[1]}")
print(f"Classes: {len(np.unique(y))}")

# Build model - notice how easy it is!
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(20,)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    
    layers.Dense(16, activation='relu'),
    
    layers.Dense(3, activation='softmax')
])

# Compile with advanced features
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\nModel compiled with:")
print("  • Adam optimizer (adaptive learning rate)")
print("  • Categorical crossentropy loss")
print("  • Multiple metrics tracking")
print("  • Batch normalization")
print("  • Dropout regularization")

# Train with callbacks
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

print("\nTraining with early stopping...")
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=0
)

print(f"Training stopped at epoch {len(history.history['loss'])}")

# Evaluate
test_loss, test_acc = model.evaluate(
    X_test, y_test, verbose=0
)

print(f"\nTest Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_acc:.2%}")

print("\nComplete deep learning pipeline in < 30 lines of code!")